In [186]:
import pandas as pd
import numpy as np
from datetime import datetime
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Processing data

In [107]:
hurricane_list = []
with open('hurdat2.txt') as fp:
    line = fp.readline()
    while line:
        if 'AL' in line:
            storm_id = line.split(',')[0].strip()
            storm_name = line.split(',')[1].strip()
        else:
            text = line.split(',')
            oldformat = text[0] + text[1]
            datetimeobject = datetime.strptime(oldformat,'%Y%m%d %H%M%S')
            date = datetimeobject.strftime('%m-%d-%Y %H%M%S')
            year = text[0].strip()[:4]
            month = text[0].strip()[4:6]
            landfall = text[2]
            status = text[3].strip()
            latitude = text[4].strip()
            longitude = text[5].strip()
            wind = text[6].strip()
            hurricane_list.append(
                {
                    "storm_id": storm_id, 
                    "name": storm_name,
                    "date": date,
                    "year": year,
                    "month": month,
                    "landfall": landfall,
                    "status" : status,
                    "latitude" : latitude,
                    "longitude" : longitude,
                    "windspeed" : wind,
                    "pressure" : pressure
                }) 
        line = fp.readline() 


In [108]:
# Create Dataframe and reorder columns
df = pd.DataFrame(hurricane_list)
df = df.loc[:, ["storm_id", "name","date", "year", "month", "status", "latitude",
                                    "longitude", "windspeed", "pressure", "landfall"]]

In [109]:
df.shape

(53501, 11)

In [110]:
df.head()

,storm_id,name,date,year,month,status,latitude,longitude,windspeed,pressure,landfall
0,AL011851,UNNAMED,06-25-1851 000000,1851,06,HU,28.0N,94.8W,80,-999,
1,AL011851,UNNAMED,06-25-1851 060000,1851,06,HU,28.0N,95.4W,80,-999,
2,AL011851,UNNAMED,06-25-1851 120000,1851,06,HU,28.0N,96.0W,80,-999,
3,AL011851,UNNAMED,06-25-1851 180000,1851,06,HU,28.1N,96.5W,80,-999,
4,AL011851,UNNAMED,06-25-1851 210000,1851,06,HU,28.2N,96.8W,80,-999,L


In [111]:
df.isna().sum()

storm_id     0
name         0
date         0
year         0
month        0
status       0
latitude     0
longitude    0
windspeed    0
pressure     0
landfall     0
dtype: int64

In [113]:
df.dtypes

storm_id     object
name         object
date         object
year         object
month        object
status       object
latitude     object
longitude    object
windspeed    object
pressure     object
landfall     object
dtype: object

### Change types for Windspeed and Pressure to numeric and datetime

#### Converting windspeed from kt to mph

In [115]:
df['windspeed'] = pd.to_numeric(df['windspeed'])
df['windspeed'] = df['windspeed'] * 1.15078
df['pressure'] = pd.to_numeric(df['pressure'])
df['date'] = pd.to_datetime(df['date'])

### Getting min and max date

In [143]:
new_df = df.groupby('storm_id')
new_df = new_df['date']
start_date = new_df.min()
end_date = new_df.max()
duration = end_date - start_date


### Merge Start Date, End Date, and Duration to one dataframe. 

In [161]:
new_df = pd.DataFrame({"start_date": start_date
                             ,"end_date": end_date
                             ,"duration" : duration
                            }).reset_index()

In [162]:
new_df.head()

,storm_id,start_date,end_date,duration
0,AL011851,1851-06-25 00:00:00,1851-06-28 00:00:00,3 days 00:00:00
1,AL011852,1852-08-19 00:00:00,1852-08-30 00:00:00,11 days 00:00:00
2,AL011853,1853-08-05 12:00:00,1853-08-05 12:00:00,0 days 00:00:00
3,AL011854,1854-06-25 00:00:00,1854-06-27 12:00:00,2 days 12:00:00
4,AL011855,1855-08-06 12:00:00,1855-08-06 12:00:00,0 days 00:00:00


In [160]:
pd.merge(date_df.head(100), df.head(100)[['latitude', 'date']], left_on = 'start_date', right_on = 'date')

0    28.0N
1    20.5N
Name: latitude, dtype: object

In [165]:
new_df['begin_lat'] = pd.merge(new_df, df[['latitude', 'date']], left_on = 'start_date', right_on = 'date')['latitude']
new_df['begin_lon'] = pd.merge(new_df, df[['longitude', 'date']], left_on = 'start_date', right_on = 'date')['longitude']
new_df['end_lat'] = pd.merge(new_df, df[['latitude', 'date']], left_on = 'end_date', right_on = 'date')['latitude']
new_df['end_lon'] = pd.merge(new_df, df[['longitude', 'date']], left_on = 'end_date', right_on = 'date')['longitude']

In [166]:
new_df.head()

,storm_id,start_date,end_date,duration,begin_lat,begin_lon,end_lat,end_lon
0,AL011851,1851-06-25 00:00:00,1851-06-28 00:00:00,3 days 00:00:00,28.0N,94.8W,31.0N,100.2W
1,AL011852,1852-08-19 00:00:00,1852-08-30 00:00:00,11 days 00:00:00,20.5N,67.1W,41.0N,68.0W
2,AL011853,1853-08-05 12:00:00,1853-08-05 12:00:00,0 days 00:00:00,32.5N,69.0W,32.5N,69.0W
3,AL011854,1854-06-25 00:00:00,1854-06-27 12:00:00,2 days 12:00:00,26.0N,92.5W,28.0N,103.0W
4,AL011855,1855-08-06 12:00:00,1855-08-06 12:00:00,0 days 00:00:00,22.2N,98.0W,22.2N,98.0W


### Getting max windspeed

In [169]:
wind_df = df.groupby('storm_id').windspeed.max()

In [172]:
new_df = pd.merge(new_df, wind_df, on = 'storm_id')

In [173]:
new_df.head()

,storm_id,start_date,end_date,duration,begin_lat,begin_lon,end_lat,end_lon,windspeed
0,AL011851,1851-06-25 00:00:00,1851-06-28 00:00:00,3 days 00:00:00,28.0N,94.8W,31.0N,100.2W,92.0624
1,AL011852,1852-08-19 00:00:00,1852-08-30 00:00:00,11 days 00:00:00,20.5N,67.1W,41.0N,68.0W,115.0780
2,AL011853,1853-08-05 12:00:00,1853-08-05 12:00:00,0 days 00:00:00,32.5N,69.0W,32.5N,69.0W,57.5390
3,AL011854,1854-06-25 00:00:00,1854-06-27 12:00:00,2 days 12:00:00,26.0N,92.5W,28.0N,103.0W,80.5546
4,AL011855,1855-08-06 12:00:00,1855-08-06 12:00:00,0 days 00:00:00,22.2N,98.0W,22.2N,98.0W,103.5702


### Getting max pressure

In [175]:
pressure_df = df.groupby('storm_id').pressure.max()
new_df = pd.merge(new_df, pressure_df, on = 'storm_id')

In [176]:
new_df.head()

,storm_id,start_date,end_date,duration,begin_lat,begin_lon,end_lat,end_lon,windspeed,pressure
0,AL011851,1851-06-25 00:00:00,1851-06-28 00:00:00,3 days 00:00:00,28.0N,94.8W,31.0N,100.2W,92.0624,-999
1,AL011852,1852-08-19 00:00:00,1852-08-30 00:00:00,11 days 00:00:00,20.5N,67.1W,41.0N,68.0W,115.0780,-999
2,AL011853,1853-08-05 12:00:00,1853-08-05 12:00:00,0 days 00:00:00,32.5N,69.0W,32.5N,69.0W,57.5390,-999
3,AL011854,1854-06-25 00:00:00,1854-06-27 12:00:00,2 days 12:00:00,26.0N,92.5W,28.0N,103.0W,80.5546,-999
4,AL011855,1855-08-06 12:00:00,1855-08-06 12:00:00,0 days 00:00:00,22.2N,98.0W,22.2N,98.0W,103.5702,-999


In [177]:
new_df.shape

(1936, 10)

### Getting name in new df

In [184]:
new_df['name'] = pd.merge(new_df, df[['storm_id', 'name']], on = 'storm_id')['name']

In [185]:
new_df.head()

,storm_id,start_date,end_date,duration,begin_lat,begin_lon,end_lat,end_lon,windspeed,pressure,name
0,AL011851,1851-06-25 00:00:00,1851-06-28 00:00:00,3 days 00:00:00,28.0N,94.8W,31.0N,100.2W,92.0624,-999,UNNAMED
1,AL011852,1852-08-19 00:00:00,1852-08-30 00:00:00,11 days 00:00:00,20.5N,67.1W,41.0N,68.0W,115.0780,-999,UNNAMED
2,AL011853,1853-08-05 12:00:00,1853-08-05 12:00:00,0 days 00:00:00,32.5N,69.0W,32.5N,69.0W,57.5390,-999,UNNAMED
3,AL011854,1854-06-25 00:00:00,1854-06-27 12:00:00,2 days 12:00:00,26.0N,92.5W,28.0N,103.0W,80.5546,-999,UNNAMED
4,AL011855,1855-08-06 12:00:00,1855-08-06 12:00:00,0 days 00:00:00,22.2N,98.0W,22.2N,98.0W,103.5702,-999,UNNAMED


In [208]:
new_df['start_date'] = new_df['start_date'].dt.date
new_df['end_date'] = new_df['end_date'].dt.date

In [209]:
new_df.head()

,storm_id,start_date,end_date,duration,begin_lat,begin_lon,end_lat,end_lon,windspeed,pressure,name
0,AL011851,1851-06-25,1851-06-28,3 days 00:00:00,28.0N,94.8W,31.0N,100.2W,92.0624,-999,UNNAMED
1,AL011852,1852-08-19,1852-08-30,11 days 00:00:00,20.5N,67.1W,41.0N,68.0W,115.0780,-999,UNNAMED
2,AL011853,1853-08-05,1853-08-05,0 days 00:00:00,32.5N,69.0W,32.5N,69.0W,57.5390,-999,UNNAMED
3,AL011854,1854-06-25,1854-06-27,2 days 12:00:00,26.0N,92.5W,28.0N,103.0W,80.5546,-999,UNNAMED
4,AL011855,1855-08-06,1855-08-06,0 days 00:00:00,22.2N,98.0W,22.2N,98.0W,103.5702,-999,UNNAMED


# New File to get damage caused by hurricanes

In [214]:
# Loading first file
cols = [ 'EVENT_TYPE', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'BEGIN_DATE_TIME', 'END_DATE_TIME']
df = pd.read_csv('https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d1950_c20210803.csv.gz',
                 compression = 'gzip', usecols= cols)
df = df[df['EVENT_TYPE'] == 'Hurricane']

# Using a loop to download and concatenate other files with same pattern names
years1 = list(range(1951, 1972))
years2 = list(range(1972, 2014))

# First types of files
for i in range(len(years1)):
    temp_df = pd.read_csv('https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d' + str(years1[i]) +'_c20210803.csv.gz', 
                          compression = 'gzip', low_memory=False, usecols= cols)
    temp_df = temp_df[temp_df['EVENT_TYPE'] == 'Hurricane']
    df = pd.concat([df, temp_df], ignore_index=True)
    
# Second type of files
for i in range(len(years2)):
    temp_df = pd.read_csv('https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d' + str(years2[i]) +'_c20220425.csv.gz',
                          compression = 'gzip', low_memory=False, usecols= cols)
    temp_df = temp_df[temp_df['EVENT_TYPE'] == 'Hurricane']
    df = pd.concat([df, temp_df], ignore_index=True)
    
# Files without a pattern in the names are downloaded and concatenated separately
df = pd.concat([df, pd.read_csv('https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2014_c20220719.csv.gz', compression = 'gzip', low_memory=False, usecols= cols)], ignore_index=True)
df = pd.concat([df, pd.read_csv('https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2015_c20220425.csv.gz', compression = 'gzip', low_memory=False, usecols= cols)], ignore_index=True)
df = pd.concat([df, pd.read_csv('https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2016_c20220719.csv.gz', compression = 'gzip', low_memory=False, usecols= cols)], ignore_index=True)
df = pd.concat([df, pd.read_csv('https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2017_c20220719.csv.gz', compression = 'gzip', low_memory=False, usecols= cols)], ignore_index=True)
df = pd.concat([df, pd.read_csv('https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2018_c20220425.csv.gz', compression = 'gzip', low_memory=False, usecols= cols)], ignore_index=True)
df = pd.concat([df, pd.read_csv('https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2019_c20220425.csv.gz', compression = 'gzip', low_memory=False, usecols= cols)], ignore_index=True)
df = pd.concat([df, pd.read_csv('https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2020_c20220816.csv.gz', compression = 'gzip', low_memory=False, usecols= cols)], ignore_index=True)
df = pd.concat([df, pd.read_csv('https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2021_c20220921.csv.gz', compression = 'gzip', low_memory=False, usecols= cols)], ignore_index=True)
df = pd.concat([df, pd.read_csv('https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2022_c20220921.csv.gz', compression = 'gzip', low_memory=False, usecols= cols)], ignore_index=True)
df = df[df['EVENT_TYPE'] == 'Hurricane']
df.reset_index(drop = True, inplace = True)

In [216]:
df.BEGIN_DATE_TIME = pd.to_datetime(df['BEGIN_DATE_TIME']).dt.date
df.END_DATE_TIME = pd.to_datetime(df['END_DATE_TIME']).dt.date

In [217]:
df.dtypes

EVENT_TYPE           object
BEGIN_DATE_TIME      object
END_DATE_TIME        object
INJURIES_DIRECT       int64
INJURIES_INDIRECT     int64
DEATHS_DIRECT         int64
DEATHS_INDIRECT       int64
DAMAGE_PROPERTY      object
DAMAGE_CROPS         object
dtype: object

In [ ]:
df.head()

In [218]:
pd.merge(new_df, df, left_on = 'start_date', right_on = 'BEGIN_DATE_TIME')

,storm_id,start_date,end_date,duration,begin_lat,begin_lon,end_lat,end_lon,windspeed,pressure,name,EVENT_TYPE,BEGIN_DATE_TIME,END_DATE_TIME,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS
